### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

I start as before with our message class, defining this thing. It's the sort of analogy to the state with Line Graph, although it's to describe how we interact between our agents. 

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

and now I mentioned that the distributed runtime consists of these two things. One of them is the host, and this is how you create the host, autogen-x-runtimes-grpc, the grpc-worker-agent-runtime-host. So this is something which uses grpc, the remote procedure technique, to be able to send messages, and this host will run on my local host on port 50051, and I will start it with this. So this is now going to be a running host, and grpc, of course, is a cross-language approach for sending function calls between different languages, and it's super powerful. It's used in many different places you can think of it. It's like making REST HTTP calls, except you're able to call directly from one function to another, and grpc is used all over the place where interactive messaging needs to be implemented that crosses process boundaries. So that is grpc. We've started a host, and it is running, 

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

what we're now going to do is, first of all, reintroduce an old friend. We're going to bring back the autogen-server tool using our server search API, and the reason why the way we're doing it is through, we're doing it through LangChain, so we're introducing a few things from before. So we create the Google server API wrapper. We create a LangChain tool for internet search. This is the same tool that we used last week, and now we wrap that in a LangChain tool wrapper so that it becomes an autogen tool, and there it is, an autogen tool for searching the internet using the server API.

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

So originally, I was going to have this play rock-paper-scissors in a distributed way, but I realized it was a bit frivolous, and we should be trying to at least have some commercial footing here. So I was going to make it do the stock price comparison, and I thought, you know, we've already done some of them, and we've got a whole week coming on that. So instead, I've gone with this. We're trying to make a business decision, and let's say that business decision is whether we should use autogen in a new AI agent project, and we want to have two different agents. One agent research the pros of autogen by using web searches, and the other agent research the cons, the negatives, the drawbacks. And so the two agents will go off and do their analysis, do their searching, and then they will come together, and we will have a judge agent that must make a decision whether to use autogen for a project, and it must be based purely on research from its team, from its agent team respond with the decision brief rationale. So this is sort of by analogy with the rock-paper-scissors. We've got instruction one for player one, instruction two for player two, and the judge that will make the call.

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

All right, and so now we have our agents, and this is again by analogy with last time. It's really very similar.

`Player one`, I've kept calling it player one because, you know, this really is a copy-paste job basically, except I'm using gpt40mini for both because I thought it wasn't fair to use a llama. We didn't get the same quality.

So we've got gpt40mini, and so we are, this is the player one rooted agent, and we are passing in here, that we are using the `autogen_serper` tool. So that is what we're doing.

```py
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)
```

I realize even looking at this that we don't actually need a player one and player two class. This could be done in a simpler way with just one. So that's an obvious point. But anyways, for whatever reason, we've got two agents here, two different types of agents, but we're going to prompt it whether it should find the pros or cons.

But you could imagine you could switch in a different model here if you wanted to have DeepSeq do one of the research. So I'll keep it as two separate agents in case you choose to do that.

Okay, so other than that, this is exactly the same. Other than supplying the search tool, reflecting on tool use, everything here is the same.

It delegates to its underlying LLM, 

```py
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
```

and the judge is the same. It also has a delegate, an underlying LLM, and in its message handler, the method that's decorated message handler, it collects the two messages that we set up above.

It finds the two agents, player one and player two. It uses this lookup, and then it calls send message.

And this code is identical. This is all exactly the same as the code we just used yesterday with the runtime that was local.

```py
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
```

And so the reason I do this, I want to show you that without changing anything, I didn't, there's nothing here about it being distributed. It doesn't know that it's distributed.

This could still be doing rock, paper, scissors. It would be the same thing. We're just calling self.send message.

And what we don't realize is that this is running remotely, it's running on a runtime, it's running on a port here, and it's going to be the autogen core is going to be handling, calling the right function in the right agent.

So these calls that appear to just be simply, I'm calling send message right here, I call send message, and that is going to result in this `async def handle_my_message_type(..)` getting called.

And before, that was just directly like some if statements in Python that were just making that call. Now, this is going to be happening using gRPC remotely orchestrated by this distributed runtime.

But that is completely unknown to us. As far as we're concerned, we're just doing exactly the same thing. And that is the power of autogen core distributed, that we don't have to worry about the fact that these are different processes running, and they could be written in different computer programming languages. And all of the stitching together of messages is happening for us just based on looking up player one and player two, everything is happening.




In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)




Everything is happening, so enough prattle. Let's run that.

Okay, and this is where the meat happens. So I've got two different implementations that I want to show you. And we're starting with all-in-one worker.


And here's how it works.
We, first of all, say that we want to create a new worker agent runtime. And we point it at our host. This is the host. So this will be a new runtime connecting to that host. And we will start that worker.

```py
worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
await worker.start()
```


We are then going to register three agents with that worker. Agent 1, Player 1, Player 2, and the Judge. Here they are, Player 1, Player 2, and the Judge, all being registered with this worker, with this glpc worker agent runtime at that host.

```py
await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
await Judge.register(worker, "judge", lambda: Judge("judge"))
```

And there we go. There is Player 1, Player 2, and the Judge. These are the factories that will create them.


And now we collect the agent ID of the Judge.

```py
agent_id = AgentId("judge", "default")
```


So I'm going to run this, and because this is set to true, only this code here is going to run. And it's done.

```py
if ALL_IN_ONE_WORKER:
    # ...código anterior...
else:
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")
```


And now, this is the same as before. It's just the same thing. We're going to send Go, the Go message, to the agent ID that I set here, to the Judge. And let's see what happens. So it's thinking.


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)


In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen in your AI Agent project:

1. **Scalability**: AutoGen supports the development of scalable systems due to its modular and extensible framework. This allows you to easily expand and adapt the system as your needs grow.

2. **Ease of Use**: AutoGen comes with integrated observability and debugging tools. This enhances monitoring and control over agent workflows, making it easier to manage and troubleshoot your AI agents.

Overall, these features make AutoGen a compelling choice for developing effective and efficient AI agents.

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **Not Beginner-Friendly**: AutoGen may have a steep learning curve for newcomers, making it less accessible for teams without extensive experience in AI development.

2. **Documentation Challenges**: There are issues with the consistency and clarity of the documentation, which can hinder effective implementation and troubleshooting.

3. **Manual Orchestration Required**: AutoGen necessitates manual orchestration, which can increase the complexity of project management and lead to potential errors during the integration of different components.

Considering these factors is important in making an informed decision for your project. 

TERMINATE



## Decision:

Based on the provided pros and cons of AutoGen, my decision is to **use AutoGen for the project**. 

The benefits of scalability and ease of use, particularly with the integrated observability and debugging tools, outweigh the challenges posed by the learning curve and documentation issues. While the manual orchestration requirement does introduce complexity, the overall advantages in managing and expanding the AI agents effectively make AutoGen a suitable choice for our needs.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()



## put all on the more worker as false

And very briefly, as I promised, I want to show you what it looks like if we restart everything and now we go through and we put all on the more worker as false. What does this mean exactly? So we are going to of course have the same message, the same host that we start running on localize to 5.0.5.1, and we use the same tools, the same instructions, we make the same agents, our agent code isn't touched, but now it's going to be managed differently.


**2. Preparation (Identical in Both Modes)**

We're now going to run this and we're going to be following this line right here, and this is actually going to create three different runtimes that I'm calling worker one, worker two, and worker, and it's worker for the third one, the judge's one. And we're going to start each of these three glpc worker runtimes, and then player one, we're going to register with worker one, player two with worker two, and the judge with just the one that's just called worker. And so that is happening.

```python
worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
await worker1.start()
await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
await worker2.start()
await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
await worker.start()
await Judge.register(worker, "judge", lambda: Judge("judge"))
agent_id = AgentId("judge", "default")
```



**3. Conceptual Difference Between Modes**

And so what's the difference here? What we're saying is that instead of having our three agents running in a remote worker on our remote host, we now have three workers, and the three workers each running one of our agents. So they're now in different runtimes. They're all on the same host, but you could imagine that this is now, they're completely separate. This is something where things are interacting between runtimes.



**4. Execution and Results**

And so now we send the same go message, and presumably very similar messages, and now crossing the ether, one agent is calling gpc4rmini for the pros, one agent for the cons, and then as a result of that, they will come back to the judge, and the judge has come. We see pretty similar looking pros, the community and support is a new one, I think. Scalability is up at first. The cons, limited customization, that seems to slightly fly in the face of flexibility. I guess our agents disagree. Performance variability, cost considerations, and ethical content control, okay. But nonetheless, the recommendation is to proceed. That is the view of even when we distribute across multiple workers, it does still maintain that viewpoint. So there we go.

```python
response = await worker.send_message(Message(content="Go!"), agent_id)
display(Markdown(response.content))

await worker.stop()
await worker1.stop()
await worker2.stop()
await host.stop()
```

**5. Conclusion and Architectural Reflection**

That's the example I wanted to show you. I know we've gone through it quickly, but it's just to get that sense that the powerful thing about this is that without changing your code with the same definitions of your classes, these can be running in different configurations and different kinds of runtimes, and basically what Autogen is doing is it's handling message calling across process boundaries as transparently as if these were just simply classes with methods calling each other directly in Python code. That abstraction is what it's doing, and when you think about a future where potentially there could be millions or maybe even billions of agents interacting all over the place, what Microsoft is doing is putting their stake in the ground for this is a sort of playpen. This is a world where agents can live and interact. You just put your agent code within this wrapper, this agent wrapper. You make your versions of message so that you declare the way that we have, and then your agents can interact with each other no matter where they are in the world and no matter what programming language they're written in.


And with that, that brings us to the end.



In [12]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False


@dataclass
class Message:
    content: str

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 


serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)
    
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")


response = await worker.send_message(Message(content="Go!"), agent_id)
display(Markdown(response.content))


await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

await host.stop()

## Pros of AutoGen:
Here are several reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Customizability**: AutoGen allows developers to create content tailored to specific needs, which enhances flexibility in adapting agents for various tasks.

2. **Multi-Agent Cooperation**: The framework supports multiple agents that can communicate and collaborate, fostering divergent thinking and more robust problem-solving methods.

3. **Time Efficiency**: AutoGen enhances productivity by enabling agents to complete tasks quickly and efficiently, saving significant time in workflow management.

4. **Continuous Improvement**: Developed by Microsoft, AutoGen benefits from regular updates and enhancements, ensuring access to the latest features and improvements.

5. **Modular Design**: The framework is modular and easy to maintain, suitable for both straightforward and complex multi-agent scenarios, which simplifies development and scaling.

6. **Enhanced Reasoning and Factuality**: Using AutoGen can improve the reasoning abilities and factual accuracy of AI agents, leading to better performance in decision-making tasks.

These advantages make AutoGen a compelling choice for building adaptive and efficient AI agent systems. 

TERMINATE

## Cons of AutoGen:
Here are some key cons of using AutoGen in your AI Agent project:

1. **Poor Documentation**: Many users find the documentation difficult to read and lacking in sufficient examples, making it challenging to properly implement the framework.

2. **Functionality Issues**: There are reports of certain features, such as structured outputs, not functioning as intended, which could hinder development.

3. **Lack of Differentiation**: AutoGen has not clearly differentiated itself from similar applications, which may lead to confusion about its unique benefits and features.

4. **Potential Bugs**: Users have experienced instances where some aspects of the software do not work as promised, raising concerns about reliability.

5. **Steep Learning Curve**: Due to the documentation quality and complexity, developers may face a steep learning curve when trying to adopt AutoGen.

These factors could potentially complicate your project depending on your specific needs and team expertise. 

TERMINATE



## Decision:

Based on the research provided by the team, I recommend using AutoGen for the project. The pros, particularly the customizability, multi-agent cooperation, and time efficiency, outweigh the cons related to documentation and potential functionality issues. The capability for enhanced reasoning and continuous improvement from Microsoft is particularly advantageous for developing effective AI agents. While the steep learning curve and poor documentation are valid concerns, the overall benefits of versatility and performance enhancement make AutoGen a strong candidate for the project's needs.

TERMINATE